# Pressure data extraction

In [19]:
#import libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

In [20]:
time_stamps = []
node_list = []
pressure_list = []


In [21]:
def extract_data(path):
    global time_stamps
    global node_list
    global pressure_list
    with open(path, 'r') as file:
        unfiltered_data = file.read()

    unfiltered_data = unfiltered_data.split('[')
    filtered_data = []

    for value in unfiltered_data:
        filtered_data.append(value.split(']'))

    filtered_data = filtered_data[1:]
    final_data = []
    
    for i in range(1, len(filtered_data), 4):
        for value in filtered_data[i]: 
            if value.strip():
                node_values = re.findall(r'\d+', value)
                node = [int(x) for x in node_values]
                node_list.append(node)

        for value in filtered_data[i+1]:
            if value.strip():
                pressure_values = re.findall(r'\d+', value)
                pressure = [int(x) for x in pressure_values]
                pressure_list.append(pressure)

        for value in filtered_data[i+2]:
            value = value.strip()
            
            if (i+2 == len(filtered_data)-1):
                value = value.split('\n')[0]
            if value:
                time = float(value)
                time_stamps.append(time)

        extract = list(zip(node, pressure))
        final_data.append([extract, time])

    #print(time_stamps)
    return final_data


In [22]:
path = "240523 ctl_5-40\S5_5N_2500PPS_18"
dat = extract_data(path)

In [23]:
time_diffs = [time_stamps[i+1] - time_stamps[i] for i in range(len(time_stamps)-1)]
print(time_diffs)

average_time_diff = sum(time_diffs) / len(time_diffs)

frequency = 1 / average_time_diff

print("Frequency in Hz:", frequency)


[0.04302668571472168, 0.043005943298339844, 0.04310941696166992, 0.04292583465576172, 0.042922019958496094, 0.04296088218688965, 0.042964935302734375, 0.04303908348083496, 0.043012380599975586, 0.04301142692565918, 0.04299044609069824, 0.04301595687866211, 0.04300689697265625, 0.043023109436035156, 0.04295015335083008, 0.043022871017456055, 0.042995452880859375, 0.042990922927856445, 0.04300832748413086, 0.04306221008300781, 0.042952537536621094, 0.04303455352783203, 0.043025970458984375, 0.04300880432128906, 0.0430140495300293, 0.04297637939453125, 0.04302692413330078, 0.04300355911254883, 0.04296875, 0.04301261901855469, 0.04299354553222656, 0.042987823486328125, 0.04306173324584961, 0.042990684509277344, 0.042969465255737305, 0.04301285743713379, 0.043047428131103516, 0.0429840087890625, 0.04283928871154785, 0.043227195739746094, 0.043090105056762695, 0.042882442474365234, 0.04268145561218262, 0.043318748474121094, 0.043054819107055664, 0.042948007583618164, 0.042812347412109375, 0.

In [24]:
num_nodes = 64
time_list = [entry[1] for entry in dat]

df = pd.DataFrame(0, index = time_list, columns = range(1,num_nodes+1))

for i, [val,t] in enumerate(dat):
    for n,p in val:
        df.at[time_list[i], n] = p


In [25]:
# saving as csv to check
#df.to_csv('sensor_data.csv', header='timestamp', index_label='nodes')


## Retiming and linear interpolation

In [26]:
def custom_retime_linear(df, new_time):
    # Convert unix code to DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, unit = 's')
        
    new_time_series = pd.date_range(start = df.index[0], end = df.index[len(df)-1], freq=new_time)

    # init new df
    new_df = pd.DataFrame(index=new_time_series)

    # Linear interpol
    for column in df.columns:
        new_df[column] = np.interp(new_time_series.astype(np.int64), df.index.astype(np.int64), df[column])
        new_df = new_df.astype(int)

    return new_df


In [27]:
# retiming the df
time_difference = pd.Timedelta(seconds=0.0165)
newdf = custom_retime_linear(df, time_difference)
newdf.to_csv('sensor_data_new.csv', header='timestamp', index_label='nodes')

In [28]:
frequency = pd.infer_freq(newdf.index)
print(frequency)

16500U


In [29]:
datetime_objects = newdf.index.to_series()
differences = datetime_objects.diff().dt.total_seconds().dropna()
average_difference = differences.mean()

print("Differences:", differences.tolist())
print("Average difference:", average_difference, "seconds")
print("Frequency: ", 1/average_difference)

Differences: [0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.0165, 0.

## Mapping to RGB values

In [30]:
def map_2_rgb(x):
    if x <= 255:
        return [x, 0, 0] 
    elif x > 255 and x <= 65535:
        return [0, x//256, 0]
    else:
        return [0, 0, x//65536]
        

## Video rendering

In [31]:
fig, ax = plt.subplots()
im = ax.imshow(np.zeros((8, 8, 3), dtype=int)) 

def update(frame):
    pressure_data = newdf.iloc[frame]
    rgb_data = np.array([[map_2_rgb(value) for value in row] for row in pressure_data.values.reshape((8, 8))])
    im.set_array(rgb_data)
    return im

ani = FuncAnimation(fig, update, frames=range(len(newdf)), interval=30)
plt.axis('off')
ani.save('pressure_animation.gif', writer='pillow')
plt.close()